<a href="https://colab.research.google.com/github/yeonghun00/stock_public/blob/main/daytrading_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving 1 841 매매내역_(파일럿)_2021-07-31.xls to 1 841 매매내역_(파일럿)_2021-07-31.xls


In [2]:
import pandas as pd

df = pd.read_html(uploaded[list(uploaded)[0]])[0]

In [3]:
import datetime

df.columns = df.iloc[0]
df = df.iloc[1:]

df['거래일자'] = list(map(lambda x: datetime.datetime.strptime(x, '%m/%d %H:%M'), df['거래일자']))
df['매수/매도가'], df['수량'], df['매매금액'] = df['매수/매도가'].astype(float), df['수량'].astype(int), df['매매금액'].astype(int)

In [4]:
df = df[::-1].reset_index(drop=True)
df

,거래일자,매매구분,종목명,매수/매도가,수량,매매금액
0,1900-06-14 09:01:00,매수,유니크,8720.0,5688,49598810
1,1900-06-14 09:02:00,매도,유니크,8760.0,2309,20226840
2,1900-06-14 09:02:00,매도,유니크,8760.0,123,1077480
3,1900-06-14 09:02:00,매도,유니크,8760.0,10,87600
4,1900-06-14 09:02:00,매도,유니크,8760.0,20,175200
...,...,...,...,...,...,...
6169,1900-07-29 14:16:00,매도,한국비엔씨,16750.0,278,4656500
6170,1900-07-29 14:16:00,매도,한국비엔씨,16750.0,142,2378500
6171,1900-07-29 14:16:00,매도,한국비엔씨,16750.0,225,3768750
6172,1900-07-29 14:16:00,매도,한국비엔씨,16750.0,344,5762000


In [5]:
code_dic = {}

for i in range(len(df)):
  if df.loc[i]['종목명'] not in list(code_dic):
    code_dic[df.loc[i]['종목명']] = {'time':[df.loc[i]['거래일자']], 'type':[df.loc[i]['매매구분']], 'price':[df.loc[i]['매수/매도가']], 'quant':[df.loc[i]['수량']]}
  else:
    code_dic[df.loc[i]['종목명']]['time'].append(df.loc[i]['거래일자'])
    code_dic[df.loc[i]['종목명']]['type'].append(df.loc[i]['매매구분'])
    code_dic[df.loc[i]['종목명']]['price'].append(df.loc[i]['매수/매도가'])
    code_dic[df.loc[i]['종목명']]['quant'].append(df.loc[i]['수량'])

code_df = pd.DataFrame.from_dict(code_dic).T

In [6]:
code_df

,time,type,price,quant
유니크,"[1900-06-14 09:01:00, 1900-06-14 09:02:00, 190...","[매수, 매도, 매도, 매도, 매도, 매도, 매도, 매도, 매도]","[8720.0, 8760.0, 8760.0, 8760.0, 8760.0, 8760....","[5688, 2309, 123, 10, 20, 114, 235, 11, 2866]"
서울가스,"[1900-06-14 09:06:00, 1900-06-14 09:12:00, 190...","[매수, 매도, 매도, 매도, 매도, 매도]","[144934.0, 138500.0, 138500.0, 138500.0, 13850...","[342, 75, 2, 1, 2, 262]"
아이티센,"[1900-06-14 09:18:00, 1900-06-14 09:20:00, 190...","[매수, 매도, 매도, 매도, 매도, 매도]","[7770.0, 7650.0, 7650.0, 7650.0, 7650.0, 7530.0]","[6361, 102, 4953, 110, 1052, 144]"
티비씨,"[1900-06-14 09:41:00, 1900-06-14 09:44:00]","[매수, 매도]","[1679.0, 1690.0]","[29154, 29154]"
아주IB투자,"[1900-06-14 10:04:00, 1900-06-14 10:24:00, 190...","[매수, 매도, 매도, 매도, 매도, 매도, 매도, 매도, 매도, 매도, 매도, 매...","[7490.0, 7490.0, 7490.0, 7490.0, 7490.0, 7490....","[6613, 374, 140, 5, 16, 59, 20, 10, 50, 50, 10..."
...,...,...,...,...
OCI,"[1900-07-28 15:03:00, 1900-07-28 15:05:00]","[매수, 매도]","[121500.0, 122000.0]","[440, 440]"
KC코트렐,"[1900-07-29 09:11:00, 1900-07-29 09:11:00, 190...","[매수, 매수, 매도]","[9320.0, 9320.0, 9430.0]","[442, 100, 542]"
디스플레이텍,"[1900-07-29 10:38:00, 1900-07-29 10:38:00, 190...","[매수, 매수, 매도, 매도, 매도, 매도, 매도, 매도, 매도, 매도, 매도]","[5192.0, 5200.0, 5291.0, 5290.0, 5330.0, 5330....","[10456, 10456, 10378, 78, 28, 5200, 3173, 454,..."
KT서브마린,"[1900-07-29 12:47:00, 1900-07-29 13:24:00, 190...","[매수, 매도, 매도, 매도, 매도, 매도, 매도, 매도, 매도, 매도, 매도, 매...","[7321.0, 7300.0, 7300.0, 7300.0, 7300.0, 7300....","[7472, 83, 10, 1, 1, 283, 25, 440, 1, 2, 10, 3..."


In [73]:
def get_profit(code):
  q = 0
  profit = 0
  av_buy = 0
  av_sell = 0
  for t in [x for x in list(zip(code_df.loc[code]['type'], code_df.loc[code]['price'], code_df.loc[code]['quant']))]:
    if t[0] == '매수':
      q += t[2]
      profit -= t[1] * t[2]
      av_buy += t[1]
    else:
      q -= t[2]
      profit += t[1] * t[2]
      av_sell += t[1]
  av_buy /= code_df.loc[code]['type'].count('매수')
  av_sell /= code_df.loc[code]['type'].count('매도')

  if q != 0: return 0, 1
  return profit, av_sell/av_buy

In [88]:
codes = {}
total_profit = 0
total_per = 1

for code in code_df.index:
  total_profit += get_profit(code)[0]
  total_per *= get_profit(code)[1]
  codes[code] = [get_profit(code)[0], get_profit(code)[1]]

In [89]:
total_per

7.009841757318611

In [90]:
total_profit

352696550.0

In [102]:
pd.DataFrame.from_dict(codes, orient='index', columns=['profit', 'per']).sort_values(by = 'profit')

,profit,per
인터파크,-10150952.0,0.953174
코나아이,-5068546.0,0.976060
혜인,-3423480.0,0.938076
올리패스,-3292050.0,0.938406
광림,-3273313.0,0.983151
...,...,...
박셀바이오,10369778.0,1.009985
이루온,21424024.0,0.993408
일신석재,24896662.0,1.045190
SBI인베스트먼트,26443192.0,1.032400


In [103]:
code_df.loc['서린바이오']

time     [1900-07-01 09:51:00, 1900-07-01 09:51:00, 190...
type     [매수, 매도, 매수, 매수, 매수, 매수, 매수, 매도, 매도, 매도, 매도, 매...
price    [24000.0, 23820.0, 25100.0, 25111.0, 25161.0, ...
quant    [2254, 2254, 2161, 2161, 4322, 2161, 2161, 509...
Name: 서린바이오, dtype: object